In [ ]:
import pickle

In [ ]:


with open('../../assets/traditional_assets/cv_set.pkl', 'wb') as fout:
    pickle.dump((cv_vec,, cv_model), fout)

with open('../../assets/traditional_assets/tfidf_set.pkl', 'wb') as fout:
    pickle.dump((tfidf_vec,, tfidf_model), fout)


In [ ]:


with open('../../assets/traditional_assets/cv_set.pkl', 'rb') as fout:
    pickle.dump((cv_vec,, cv_model), fout)

with open('../../assets/traditional_assets/tfidf_set.pkl', 'rb') as fout:
    pickle.dump((tfidf_vec,, tfidf_model), fout)
